In [ ]:
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import math
import matplotlib.pyplot as plt
import os
import numpy as np

FOLDER = "./Data/"

VAE_MODEL = FOLDER + "Vae_model_dict_50EPOCHS_v2"
DIGIT_SAVE_DIR_GAN_BCE = FOLDER + 'sample_digits_GAN_BCE.png'
DIGIT_SAVE_DIR_GAN_WASSERSTEIN = FOLDER + 'sample_digits_GAN_Wasserstein.png'
DIGIT_SAVE_DIR_VAE = FOLDER + 'sample_digits.png'
G_BCE = FOLDER + "Generator_200epochsBCE"
G_WASSERSTEIN = FOLDER + "Generator_200epochsWasserstein"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
z_dim = 4
z = torch.randn(100, z_dim).to(device)

In [ ]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim, loss_function):
        super(VAE, self).__init__()
        self.loss_function = loss_function
        self.zdim = z_dim
        # encoder part
        self.lstm1 = nn.LSTM(input_size=x_dim, hidden_size=h_dim1)
        self.fc31 = nn.Linear(h_dim1, z_dim)
        self.fc32 = nn.Linear(h_dim1, z_dim)
        # decoder part
        self.tconv2d1 = nn.ConvTranspose2d(1, 32, 8)
        self.tconv2d2 = nn.ConvTranspose2d(32, 16, 4)
        self.tconv2d3 = nn.ConvTranspose2d(16, 8, 2)
        self.fc2 = nn.Linear(h_dim2, x_dim)
        
    def encoder(self, x):
        h, hidden = self.lstm1(x)
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = torch.reshape(z, (-1, 1, int(math.sqrt(self.zdim)), int(math.sqrt(self.zdim)),))
        h = F.relu(self.tconv2d1(h))
        #h = torch.nn.functional.dropout(h, p=0.8)
        h = F.relu(self.tconv2d2(h))
        #h = torch.nn.functional.dropout(h, p=0.8)
        h = F.relu(self.tconv2d3(h))
        #h = torch.nn.functional.dropout(h, p=0.8)
        h = torch.flatten(h, start_dim=1)
        return F.sigmoid(self.fc2(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var
    
    def train_data(self, epoch, train_loader):
        self.train()
        train_loss = 0
        train_kld = 0
        loss_history = []
        KLD_history = []
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            
            recon_batch, mu, log_var = vae(data)
            loss, KLD= self.loss_function(recon_batch, data, mu, log_var)
            loss.backward()
            train_loss += loss.item()
            train_kld += KLD.item()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item() / len(data)))
        
        print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))
        return train_loss / len(train_loader.dataset), train_kld / len(train_loader.dataset)
    
    def test_data(self, test_loader):
        self.eval()
        test_loss= 0
        kld_loss = 0
        with torch.no_grad():
            for data, _ in test_loader:
                data = data.to(device)
                recon, mu, log_var = vae(data)
                
                # sum up batch loss
                loss, kld = self.loss_function(recon, data, mu, log_var)
                test_loss += loss.item()
                kld_loss += kld.item()
        test_loss /= len(test_loader.dataset)
        kld_loss /= len(test_loader.dataset)
        print('====> Test set loss: {:.4f}'.format(test_loss))
        return test_loss, kld_loss

# build model
vae = VAE(x_dim=784, h_dim1= 128, h_dim2=1352, z_dim=z_dim, loss_function=nn.BCELoss())
vae.load_state_dict(torch.load(VAE_MODEL, map_location=device))

with torch.no_grad():
    VAE_images = vae.decoder(z).to(device).view(100, 1, 28, 28)

In [ ]:
LOSS = "BCE"
WCL = 0.01
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, g_output_dim)
    
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return torch.tanh(self.fc4(x))
    
    
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.2)
        if LOSS == "BCE":
            return torch.sigmoid(self.fc4(x))
        elif LOSS == "Wasserstein":
            return (self.fc4(x))
z_dim = 4
mnist_dim = 28 * 28
G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
G.load_state_dict(torch.load(G_BCE, map_location=device))
with torch.no_grad():
    BCE_images = G(z).view(100, 1, 28, 28)

In [ ]:
LOSS = "Wasserstein"
WCL = 0.01
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, g_output_dim)
    
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return torch.tanh(self.fc4(x))
  
    
    
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.2)
        if LOSS == "BCE":
            return torch.sigmoid(self.fc4(x))
        elif LOSS == "Wasserstein":
            return (self.fc4(x))
z_dim = 4
mnist_dim = 28 * 28
G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
G.load_state_dict(torch.load(G_WASSERSTEIN, map_location=device))
with torch.no_grad():
    WASSERSTEIN_images = G(z).view(100, 1, 28, 28)

In [ ]:
# REF: https://machinelearningmastery.com/how-to-implement-the-inception-score-from-scratch-for-evaluating-generated-images/ 
# scale an array of images to a new size
def scale_images(images, new_shape):
  images_list = list()
  for image in images:
    # resize with nearest neighbor interpolation
    new_image = resize(image, new_shape, 0)
    # store
    images_list.append(new_image)
  return asarray(images_list)

# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
  # load inception v3 model
  model = InceptionV3()
  images = images.cpu().detach().numpy()
  # enumerate splits of images/predictions
  scores = list()
  n_part = floor(images.shape[0] / n_split)
  for i in range(n_split):
    # retrieve images
    ix_start, ix_end = i * n_part, (i+1) * n_part
    subset = images[ix_start:ix_end]
    # convert from uint8 to float32
    subset = subset.astype('float32')
    # scale images to the required size
    subset = scale_images(subset, (299,299,3))
    # pre-process images, scale to [-1,1]
    subset = preprocess_input(subset)
    # predict p(y|x)
    p_yx = model.predict(subset)
    # calculate p(y)
    p_y = expand_dims(p_yx.mean(axis=0), 0)
    # calculate KL divergence using log probabilities
    kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
    # sum over classes
    sum_kl_d = kl_d.sum(axis=1)
    # average over images
    avg_kl_d = mean(sum_kl_d)
    # undo the log
    is_score = exp(avg_kl_d)
    # store
    scores.append(is_score)
    # average across images
  is_avg, is_std = mean(scores), std(scores)
  return is_avg, is_std

print("VAE Inception Score: ")
is_avg, is_std = calculate_inception_score(VAE_images)
print(f"mean: {is_avg} std: {is_std}")

print("GAN BCE Inception Score: ")
is_avg, is_std = calculate_inception_score(BCE_images)
print(f"mean: {is_avg} std: {is_std}")

print("WGAN Inception Score: ")
is_avg, is_std = calculate_inception_score(WASSERSTEIN_images)
print(f"mean: {is_avg} std: {is_std}")
